Trace File Cleaning Program

In [72]:
import pandas as pd

# Read the Excel file
file_path = "URS_Export.xlsx"
sheet_name = "Document Preflight"  # Change to the desired sheet name
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Print the first few rows of the DataFrame
print(df.head())

           ID Class       Type  \
0  UTICA-2301   NaN    Heading   
1  UTICA-2302   NaN    Heading   
2  UTICA-2303   NaN    Heading   
3  UTICA-2304   NaN  Info Item   
4  UTICA-2305   NaN    Heading   

                                         Description  Starkey Priority  \
0                                                NaN               NaN   
1                                                NaN               NaN   
2                                                NaN               NaN   
3  Info: The purpose of this document is to defin...               NaN   
4                                                NaN               NaN   

   Verification Allocation                                  Linked Work Items  \
0                      NaN  is branched from: TULUM-1, has branch: XANADU-...   
1                      NaN  is branched from: TULUM-2, has parent: UTICA-2...   
2                      NaN  is branched from: TULUM-3, has parent: UTICA-2...   
3                      NaN

In [73]:
keeper=["ID","Linked Work Items"]
def fsplit(s):
    s1 = s['Linked Work Items'].str.split(':',n=1,expand=True)
    s1.columns=['Relationship','Link']
    s1.Relationship=s1.Relationship.str.strip()
    return s.join(s1)

dfc=(
    df
    .loc[df.Type=="User Requirement"]
    .loc[df.Status=="Approved"]
    .loc[:,["ID","Linked Work Items"]]
    .set_index("ID")
    .replace({' ':" "},regex=True)
    .apply(lambda x: x.str.split(',').explode())
    .pipe(fsplit)
    .drop(columns=['Linked Work Items'],axis=1)
    .reset_index()
)
display(dfc)

,ID,Relationship,Link
0,MASTER-35630,has parent,MASTER-35629
1,MASTER-35630,is refined by,MASTER-42254
2,MASTER-35630,is refined by,MASTER-49266
3,MASTER-35630,is refined by,MASTER-49261
4,MASTER-35630,is refined by,MASTER-49260
...,...,...,...
267991,UTICA-6849,is refined by,UTICA-2495
267992,UTICA-6849,is branched from,MASTER-6650
267993,UTICA-6849,has parent,UTICA-2400 - 5.1
267994,UTICA-6849,has branch,XANADU-1150


In [74]:
cat_list = ['is refined by','is validated by']
dfc = dfc[dfc['Relationship'].isin(cat_list)]
display(dfc)



,ID,Relationship,Link
1,MASTER-35630,is refined by,MASTER-42254
2,MASTER-35630,is refined by,MASTER-49266
3,MASTER-35630,is refined by,MASTER-49261
4,MASTER-35630,is refined by,MASTER-49260
5,MASTER-35630,is refined by,MASTER-49262
...,...,...,...
267979,UTICA-6378,is refined by,UTICA-2495
267983,UTICA-6849,is refined by,UTICA-2495
267987,UTICA-6849,is refined by,UTICA-2495
267991,UTICA-6849,is refined by,UTICA-2495


In [75]:
dfc = dfc[dfc['Link'].str.contains('MASTER|UTICA', case=False)].reset_index(drop=True)
dfc = dfc.rename(columns={'ID': 'UID'})

In [76]:
display(dfc.head(10))

,UID,Relationship,Link
0,MASTER-35630,is refined by,MASTER-42254
1,MASTER-35630,is refined by,MASTER-49266
2,MASTER-35630,is refined by,MASTER-49261
3,MASTER-35630,is refined by,MASTER-49260
4,MASTER-35630,is refined by,MASTER-49262
5,MASTER-35630,is refined by,MASTER-50794
6,MASTER-35630,is refined by,MASTER-42254
7,MASTER-35630,is refined by,MASTER-49266
8,MASTER-35630,is refined by,MASTER-49261
9,MASTER-35630,is refined by,MASTER-49260


In [77]:
dfc.to_csv("URS_Cleaned.csv",index=False)
